In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd


In [2]:
scraping_url_list = ["https://www.dunyahalleri.com/category/teknoloji-bilim/page/",
                     "https://www.dunyahalleri.com/category/kultur-sanat/page"]

In [ ]:
urls = []

for i in scraping_url_list:
    for j in range(1,28):
        
        sect_url = i + str(j)
        
        html = requests.get(sect_url).text
        soup = bs(html, "html.parser")
        news = soup.findAll("div", class_ ="herald-module col-mod-main col-lg-12 col-md-12")
        
        for news_url in news:
            for k in news_url.find_all('a', href=True):
                
                urls.append(k["href"])

In [ ]:
urls[5]
len(urls)

In [ ]:
urls_df = pd.DataFrame(urls)
    

In [ ]:
urls_df = urls_df.drop_duplicates()

In [ ]:
urls_df.info()#url'lerin olduğu listeyi dataframe'e çevirip, listede birden fazla olan urlleri sildik. 
                # url sayısı 2538'ten 868'e indi böylece kod daha kısa sürecek çünkü daha az linke gideceğiz.
urls_df.head()

In [ ]:
def text_mining(url):
    try:
        html = requests.get(url).text
        soup = bs(html, "html.parser")
        news_texts = soup.findAll("div", class_ = "col-lg-12 col-md-12 col-sm-12")[0].find_all("p")
                                                                                                                                                          
        news_text_all = ""

        for i in news_texts:
            news_text_all = news_text_all + i.text
            
        if soup.find("div", class_ = "entry-headline h5").text:
            headline = soup.find("div", class_ = "entry-headline h5").text
        else:
            print("Headline yok.")

        return [headline, news_text_all]
    except IndexError:
        return None


In [ ]:
dataset = []
urls_new = urls_df.values.tolist()
del urls_new[509] # 509. index'teki url'de hata aldım, çözemediğimden listeden sildim.

for i in urls_new:
   
    dataset.append(text_mining(i[0]))

In [ ]:
len(dataset)
dataset_cleaned = list(filter(lambda x: x is not None, dataset))

In [ ]:
nlp_project_dataset = pd.DataFrame(dataset_cleaned)


In [ ]:
nlp_project_dataset.columns = ["Headline","Haber_Metni"]

In [ ]:
nlp_project_dataset.drop_duplicates()

In [ ]:
nlp_project_dataset.index = range(0, len(nlp_project_dataset))

In [ ]:
nlp_project_dataset.head()

In [ ]:
nlp_project_dataset.to_csv("nlp_project_dataset.csv")